In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import sys

ROOT_DIR = '../'
sys.path.insert(1, '../production_code/')
from constants import *

In [2]:

accidents = pd.read_csv(ROOT_DIR + ACCIDENT_DATA_GENERAL_DIR)
node = pd.read_csv(ROOT_DIR + ACCIDENT_DATA_NODE_DIR)

C:\Users\andre\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (10,13) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
node.iloc[0]

ACCIDENT_NO                         T20060002689
NODE_ID                                       22
NODE_TYPE                                      I
VICGRID94_X                          2495701.925
VICGRID94_Y                          2411599.135
LGA_NAME                               MELBOURNE
LGA_NAME_ALL                           MELBOURNE
REGION_NAME       METROPOLITAN NORTH WEST REGION
DEG_URBAN_NAME                        MELB_URBAN
Lat                                   -37.796596
Long                                  144.951197
POSTCODE_NO                                 3051
Name: 0, dtype: object

In [4]:
accidents.iloc[0]

ACCIDENT_NO                                               T20060000010
ACCIDENTDATE                                                13/01/2006
ACCIDENTTIME                            12:42:00                      
ACCIDENT_TYPE                                                        1
Accident Type Desc                              Collision with vehicle
DAY_OF_WEEK                                                          6
Day Week Description                                            Friday
DCA_CODE                                                           113
DCA Description         RIGHT NEAR (INTERSECTIONS ONLY)               
DIRECTORY                                                          MEL
EDITION                                                             40
PAGE                                                               91A
GRID_REFERENCE_X                                                     G
GRID_REFERENCE_Y                                                     7
LIGHT_

In [5]:
accidents.loc[:,'ACCIDENTDATE'] = pd.to_datetime(accidents.loc[:,'ACCIDENTDATE'], format="%d/%m/%Y")
accidents.loc[:,'DAY_OF_WEEK'] = accidents.loc[:,'ACCIDENTDATE'].dt.day_of_week

# Comparing all the data stages

by grouping and pivotting the data, any stage can be broken down into the total number of calls made each day, averaged over all days in a certain date range. The groupped by region.

This way we can make sure the average number of emergency calls made in each region per day, stays constant accross all dates, as it should

# Comparing Original

In [6]:
date_min = pd.to_datetime(TRAIN_SPLIT_MIN_DATE) #TRAIN_SPLIT_MIN_DATE
date_max = pd.to_datetime(TEST_TRAIN_SPLIT_DATE) #TEST_SPLIT_MAX_DATE
accidents.loc[:,'POLICE_ATTEND'] = accidents['POLICE_ATTEND'].apply(lambda x: 2 - x)

accidents\
    .merge(node[['ACCIDENT_NO','REGION_NAME']].drop_duplicates(), how = 'inner')\
    .drop_duplicates(subset=['ACCIDENT_NO'])\
    .query('ACCIDENTDATE >= @date_min & ACCIDENTDATE < @date_max')\
    .pivot_table(    # sums the total number of police attended for each day, splitting by region
         index = ['ACCIDENTDATE','REGION_NAME'],
         values=['POLICE_ATTEND'],
         aggfunc='sum'
         )\
    .reset_index()\
    .groupby(['REGION_NAME'])['POLICE_ATTEND'].mean() 
    # .pivot_table(    # sums the total number of police attended for each day, splitting by region
    #     index = ['ACCIDENTDATE'],
    #     columns=['REGION_NAME'],
    #     values=['POLICE_ATTEND'],
    #     aggfunc='sum'
    #     )\
    # .sort_index(ascending=False)\
    # .head(60)\
    # .mean()

REGION_NAME
                                   0.711111
EASTERN REGION                     1.898903
METROPOLITAN NORTH WEST REGION     9.724162
METROPOLITAN SOUTH EAST REGION    10.585900
NORTH EASTERN REGION               1.849175
NORTHERN REGION                    1.885603
SOUTH WESTERN REGION               2.420479
WESTERN REGION                     1.827044
Name: POLICE_ATTEND, dtype: float64

# Original with basic filtering

In [7]:
# accidents = accidents.dropna(subset=['ACCIDENTDATE'])
# accidents = accidents.dropna(subset=['ACCIDENTTIME'])
# accidents = accidents.query("LIGHT_CONDITION != 9")
accidents = accidents.query("POLICE_ATTEND != 9")   # removing when unsure if police attened or not
accidents = accidents[pd.to_datetime(accidents['ACCIDENTDATE']) > pd.to_datetime(EARLIEST_DATE)].reset_index(drop = True)

accidents\
    .merge(node[['NODE_ID','REGION_NAME']]\
        .drop_duplicates(), how = 'inner')\
    .drop_duplicates(subset=['ACCIDENT_NO'])\
    .query('ACCIDENTDATE >= @date_min & ACCIDENTDATE < @date_max')\
    .pivot_table(    # sums the total number of police attended for each day, splitting by region
         index = ['ACCIDENTDATE','REGION_NAME'],
         values=['POLICE_ATTEND'],
         aggfunc='sum'
         )\
    .reset_index()\
    .groupby(['REGION_NAME'])['POLICE_ATTEND'].mean() 


REGION_NAME
                                   0.711111
EASTERN REGION                     1.898039
METROPOLITAN NORTH WEST REGION     9.726027
METROPOLITAN SOUTH EAST REGION    10.591096
NORTH EASTERN REGION               1.849057
NORTHERN REGION                    1.885514
SOUTH WESTERN REGION               2.419331
WESTERN REGION                     1.827044
Name: POLICE_ATTEND, dtype: float64

# roughly cleaned pre merge dat

In [8]:
ROUGHLY_CLEANED_PRE_MERGE_DATA_DIR

first_clean = pd.read_csv(ROOT_DIR + ROUGHLY_CLEANED_PRE_MERGE_DATA_DIR)
first_clean.loc[:,'POLICE_ATTEND'] = first_clean['POLICE_ATTEND'].apply(lambda x: 2 - x)
first_clean.loc[:,'date'] = pd.to_datetime(first_clean.loc[:,'date']).dt.date

first_clean\
    .merge(node[['ACCIDENT_NO','REGION_NAME']]\
        .drop_duplicates(),  how = 'inner')\
    .query('date >= @date_min & date < @date_max')\
    .pivot_table(    # sums the total number of police attended for each day, splitting by region
         index = ['date','REGION_NAME'],
         values=['POLICE_ATTEND'],
         aggfunc='sum'
         )\
    .reset_index()\
    .groupby(['REGION_NAME'])['POLICE_ATTEND'].mean() 

C:\Users\andre\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\pandas\core\ops\array_ops.py:73: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  result = libops.scalar_compare(x.ravel(), y, op)


REGION_NAME
                                   0.711111
EASTERN REGION                     1.961508
METROPOLITAN NORTH WEST REGION    10.191096
METROPOLITAN SOUTH EAST REGION    10.878767
NORTH EASTERN REGION               1.867034
NORTHERN REGION                    1.936087
SOUTH WESTERN REGION               2.502545
WESTERN REGION                     1.854560
Name: POLICE_ATTEND, dtype: float64

In [9]:

node = pd.read_csv(ROOT_DIR + ACCIDENT_DATA_NODE_DIR)
first_clean = pd.read_csv(ROOT_DIR + ROUGHLY_CLEANED_PRE_MERGE_DATA_DIR)
first_clean.loc[:,'date'] = pd.to_datetime(first_clean.loc[:,'date']).dt.date

# only keeping instances with ids that exist
print("initial shape: " + str(node.shape))
node = node[node['ACCIDENT_NO'].isin(accidents['ACCIDENT_NO'])]


node = node.rename(columns = {
    'REGION_NAME': 'region', 
    'LGA_NAME':'lga', 
    'NODE_ID':'node_id',
    'Lat':'lat',
    'Long':'long'
    })
node['region'] = node['region'].astype(str)
node['lga'] = node['lga'].astype(str)
node.loc[:,'lat'] = node.loc[:,'lat'].apply(pd.to_numeric)
node.loc[:,'long'] = node.loc[:,'long'].apply(pd.to_numeric)

# removing blank regions
node = node.query('region != " "')

accidents_of_interest = ['date']
accidents_label_columns = ['police_needed']
id_columns = ['ACCIDENT_NO']
node_of_interest = ['node_id','lga','region','lat','long']


first_clean.loc[:,'police_needed'] = first_clean['POLICE_ATTEND'].apply(lambda x: 2 - x)


first_clean[id_columns + accidents_of_interest + accidents_label_columns].drop_duplicates()\
    .merge(
        node[id_columns + node_of_interest].drop_duplicates(), 
        how='inner')\
    .query('date >= @date_min & date < @date_max')\
    .pivot_table(    # sums the total number of police attended for each day, splitting by region
         index = ['date','region'],
         values=['police_needed'],
         aggfunc='sum'
         )\
    .reset_index()\
    .groupby(['region'])['police_needed'].mean() 
        

initial shape: (221797, 12)


region
EASTERN REGION                     1.961508
METROPOLITAN NORTH WEST REGION    10.191096
METROPOLITAN SOUTH EAST REGION    10.878767
NORTH EASTERN REGION               1.867034
NORTHERN REGION                    1.936087
SOUTH WESTERN REGION               2.502545
WESTERN REGION                     1.854560
Name: police_needed, dtype: float64

# merge stages

In [10]:
stage_1 = pd.read_csv( ROOT_DIR + ROUGHLY_CLEANED_MERGE_1_DATA_DIR)
stage_1.loc[:,'date'] = pd.to_datetime(stage_1.loc[:,'date']).dt.date

stage_1\
    .query('date >= @date_min & date < @date_max')\
    .pivot_table(    # sums the total number of police attended for each day, splitting by region
         index = ['date','region'],
         values=['police_needed'],
         aggfunc='sum'
         )\
    .reset_index()\
    .groupby(['region'])['police_needed'].mean() 

region
EASTERN REGION                     1.961508
METROPOLITAN NORTH WEST REGION    10.191096
METROPOLITAN SOUTH EAST REGION    10.878767
NORTH EASTERN REGION               1.867034
NORTHERN REGION                    1.936087
SOUTH WESTERN REGION               2.502545
WESTERN REGION                     1.854560
Name: police_needed, dtype: float64

In [11]:
stage_2 = pd.read_csv( ROOT_DIR + ROUGHLY_CLEANED_MERGE_2_DATA_DIR)
stage_2.loc[:,'date'] = pd.to_datetime(stage_2.loc[:,'date']).dt.date

stage_2\
    .query('date >= @date_min & date < @date_max')\
    .pivot_table(    # sums the total number of police attended for each day, splitting by region
         index = ['date','region'],
         values=['police_needed'],
         aggfunc='sum'
         )\
    .reset_index()\
    .groupby(['region'])['police_needed'].mean() 

region
EASTERN REGION                     1.961508
METROPOLITAN NORTH WEST REGION    10.191096
METROPOLITAN SOUTH EAST REGION    10.878767
NORTH EASTERN REGION               1.867034
NORTHERN REGION                    1.936087
SOUTH WESTERN REGION               2.502545
WESTERN REGION                     1.854560
Name: police_needed, dtype: float64

In [12]:
stage_3 = pd.read_csv( ROOT_DIR + ROUGHLY_CLEANED_MERGE_2_DATA_DIR)
stage_3.loc[:,'date'] = pd.to_datetime(stage_3.loc[:,'date']).dt.date

stage_3\
    .query('date >= @date_min & date < @date_max')\
    .pivot_table(    # sums the total number of police attended for each day, splitting by region
         index = ['date','region'],
         values=['police_needed'],
         aggfunc='sum'
         )\
    .reset_index()\
    .groupby(['region'])['police_needed'].mean() 

region
EASTERN REGION                     1.961508
METROPOLITAN NORTH WEST REGION    10.191096
METROPOLITAN SOUTH EAST REGION    10.878767
NORTH EASTERN REGION               1.867034
NORTHERN REGION                    1.936087
SOUTH WESTERN REGION               2.502545
WESTERN REGION                     1.854560
Name: police_needed, dtype: float64

# comparing to initial clean

In [13]:
first_clean = pd.read_csv(ROOT_DIR + ROUGHLY_CLEANED_DATA_DIR)
first_clean.loc[:,'date'] = pd.to_datetime(first_clean.loc[:,'date']).dt.date

first_clean\
    .query('date >= @date_min & date < @date_max')\
    .pivot_table(    # sums the total number of police attended for each day, splitting by region
         index = ['date','region'],
         values=['police_needed'],
         aggfunc='sum'
         )\
    .reset_index()\
    .groupby(['region'])['police_needed'].mean() 

region
EASTERN REGION                     1.970257
METROPOLITAN NORTH WEST REGION     9.905479
METROPOLITAN SOUTH EAST REGION    10.650685
NORTH EASTERN REGION               1.869984
NORTHERN REGION                    1.941693
SOUTH WESTERN REGION               2.483776
WESTERN REGION                     1.863489
Name: police_needed, dtype: float64

# comparing to further clean

In [14]:
second_clean = pd.read_csv(ROOT_DIR + FULLY_CLEANED_DATA_DIR)
second_clean.loc[:,'date'] = pd.to_datetime(first_clean.loc[:,'date']).dt.date

second_clean\
    .query('date >= @date_min & date < @date_max')\
    .pivot_table(    # sums the total number of police attended for each day, splitting by region
         index = ['date','region'],
         values=['police_needed'],
         aggfunc='sum'
         )\
    .reset_index()\
    .groupby(['region'])['police_needed'].mean() 

region
EASTERN REGION                     2.005632
METROPOLITAN NORTH WEST REGION    10.073973
METROPOLITAN SOUTH EAST REGION    10.815753
NORTH EASTERN REGION               1.902967
NORTHERN REGION                    1.986378
SOUTH WESTERN REGION               2.514349
WESTERN REGION                     1.882970
Name: police_needed, dtype: float64

# comparing to previotted

In [15]:
pre_pivot = pd.read_csv(ROOT_DIR + PREPIVOT_TRAIN_TEST_DATA_DIR)
pre_pivot.loc[:,'date'] = pd.to_datetime(pre_pivot.loc[:,'date']).dt.date

pre_pivot\
    .query('date >= @date_min & date < @date_max')\
    .pivot_table(    # sums the total number of police attended for each day, splitting by region
         index = ['date','region'],
         values=['police_needed'],
         aggfunc='sum'
         )\
    .reset_index()\
    .groupby(['region'])['police_needed'].mean() 

region
EASTERN REGION                     1.971706
METROPOLITAN NORTH WEST REGION     9.850000
METROPOLITAN SOUTH EAST REGION    10.592466
NORTH EASTERN REGION               1.864952
NORTHERN REGION                    1.947030
SOUTH WESTERN REGION               2.466421
WESTERN REGION                     1.861201
Name: police_needed, dtype: float64

## pre pivotted with scaled police

In [16]:
pre_pivot\
    .query('date >= @date_min & date < @date_max')\
    .pivot_table(    # sums the total number of police attended for each day, splitting by region
         index = ['region','sky','date'],
         values=['scaled_police'],
         aggfunc='sum'
         )\
    .reset_index()\
    .pivot_table(    # sums the total number of police attended for each day, splitting by region
         index = ['region','date'],
         values=['scaled_police'],
         aggfunc='mean'
         )\
    .reset_index()\
    .groupby(['region'])['scaled_police'].mean()

region
EASTERN REGION                     3.620674
METROPOLITAN NORTH WEST REGION    12.973935
METROPOLITAN SOUTH EAST REGION    14.244139
NORTH EASTERN REGION               3.005337
NORTHERN REGION                    3.291925
SOUTH WESTERN REGION               3.886423
WESTERN REGION                     3.106387
Name: scaled_police, dtype: float64

# comparing to final data

In [17]:
file = REGION_TESTING_DATA_DIR
file = REGION_TRAINING_DATA_DIR

test_data = pd.read_csv(ROOT_DIR + file)
test_data\
    .pivot_table(      # averaging rainging or not
        index = ['Region','Part of Day','Day of the Week'],
        values=['Police'],
        aggfunc='mean'
    )\
    .reset_index()\
    .pivot_table(      # adding together parts of day
        index = ['Region','Day of the Week'],
        values=['Police'],
        aggfunc='sum'
    )\
    .reset_index()\
    .groupby(['Region'])['Police'].mean()
# averaging each day of the week

# test_data

Region
EASTERN REGION                     1.994434
METROPOLITAN NORTH WEST REGION     9.447852
METROPOLITAN SOUTH EAST REGION    10.866339
NORTH EASTERN REGION               1.588027
NORTHERN REGION                    1.752009
SOUTH WESTERN REGION               2.412639
WESTERN REGION                     1.631280
Name: Police, dtype: float64